In [ ]:
import autogen
config_list_gpt4o = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ['gpt-4o']
    },
)

In [ ]:

## designing and deploying a smart traffic monitoring system for a city. Agents will collaborate on planning, implementation, analysis, and documentation.


gpt4o_config = {
    "config_list": [
        {
            "model": "gpt-4o",
            "api_key": "Specified my API key"
        }
    ],
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "timeout": 120,
}

admin = autogen.UserProxyAgent(
    name="CityAdmin",
    system_message="A city admin official. Interact with a city planner to discuss and approve the infrastructure development plan. No task can proceed without your approval.",
    code_execution_config=False
)

# Planner breaks down the project into steps for smart traffic monitoring
planner = autogen.AssistantAgent(
    name='CityPlanner', 
    system_message="""Urban Planner. Propose a step-by-step implementation plan for deploying a smart traffic monitoring system.
Revise your plan based on feedback from a quality control reviewer. Your plan may involve an engineer (who handles hardware/software implementation) and a data analyst (who evaluates traffic data and trends).
Clearly assign each task to either the Engineer or the Analyst, and explain your reasoning.
""",
    llm_config=gpt4o_config
)

# Engineer handles the actual implementation (hardware/software scripts)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4o_config,
    system_message="""Infrastructure Engineer. You follow the approved city plan. You write Python/shell code or system commands for implementing smart traffic sensors or dashboards.
Wrap all code in a proper code block that matches the script type. Never suggest incomplete code or require manual edits by others.
Return fully functional code only. Handle and fix any execution errors reported by the Executor. Always re-evaluate your approach if the current method fails.""",
)

# Scientist becomes a Data Analyst evaluating urban data (non-coding)
analyst = autogen.AssistantAgent(
    name="DataAnalyst",
    llm_config=gpt4o_config,
    system_message="""Data Analyst. You follow the approved infrastructure plan.
You evaluate traffic pattern data, analyze sensor logs, and generate summaries or classifications of urban traffic behavior.
You don't write code, but provide insights for the Engineer and the Report Writer based on observed data trends.""",
)

# Executor runs the code from the Engineer locally and returns the result
executor = autogen.UserProxyAgent(
    name="SystemExecutor",
    system_message="Executor. Run the implementation code provided by the Engineer (e.g., data collection, sensor simulation, visualization) and return results, including errors if any.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "smart_city_project",
        "use_docker": False,
    }
)

# Critic reviews all content — technical accuracy, feasibility, and report completeness
critic = autogen.AssistantAgent(
    name="Reviewer",
    system_message="""Project Reviewer. Review plans, implementation code, and reports for technical accuracy and feasibility.
Ensure that any final report includes real, verifiable insights, and that the steps proposed are executable within a city infrastructure context.
Flag missing elements such as diagrams, citations, or references.""",
    llm_config=gpt4o_config
)

# Writer compiles a full project report for presentation
report_writer = autogen.AssistantAgent(
    name='ReportWriter',
    system_message="""Technical Writer. Based on the insights and findings from the Data Analyst and implementation results from the Engineer, write a professional report for the city.
Sections must include: Project Overview, Implementation Plan, Data Insights, System Architecture, Results, and Recommendations.
Ensure clarity, completeness, and correct citation of resources and methods used.""",
    llm_config=gpt4o_config
)

In [17]:
groupchat = autogen.GroupChat(
    agents=[admin,planner,engineer,analyst,executor,critic,report_writer],
    messages=[],
    max_round=50
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4o_config)

In [18]:
output_report = admin.initiate_chat(manager,
                                         message="Write a 6 paragraph research report about current issues with LA traffic in United States and how can we implement smart traffic monitoring system to reduce deaths by road accidents")

CityAdmin (to chat_manager):

Write a 6 paragraph research report about current issues with LA traffic in United States and how can we implement smart traffic monitoring system to reduce deaths by road accidents

--------------------------------------------------------------------------------

Next speaker: CityPlanner

CityPlanner (to chat_manager):

**Research Report: Addressing Traffic Issues in Los Angeles and Implementing a Smart Traffic Monitoring System**

**Introduction**
Los Angeles, known for its sprawling urban landscape and vibrant culture, is also infamous for its traffic congestion and road safety challenges. The city's extensive network of highways and streets often experiences severe traffic jams, leading to increased travel times, pollution, and, most concerningly, road accidents. The high volume of vehicles, coupled with complex road networks, contributes to a significant number of traffic-related fatalities and injuries each year. Addressing these issues requires inn

In [19]:
output_report

ChatResult(chat_id=None, chat_history=[{'content': 'Write a 6 paragraph research report about current issues with LA traffic in United States and how can we implement smart traffic monitoring system to reduce deaths by road accidents', 'role': 'assistant', 'name': 'CityAdmin'}, {'content': "**Research Report: Addressing Traffic Issues in Los Angeles and Implementing a Smart Traffic Monitoring System**\n\n**Introduction**\nLos Angeles, known for its sprawling urban landscape and vibrant culture, is also infamous for its traffic congestion and road safety challenges. The city's extensive network of highways and streets often experiences severe traffic jams, leading to increased travel times, pollution, and, most concerningly, road accidents. The high volume of vehicles, coupled with complex road networks, contributes to a significant number of traffic-related fatalities and injuries each year. Addressing these issues requires innovative solutions that leverage technology to enhance traff

In [21]:
output_report.chat_history

[{'content': 'Write a 6 paragraph research report about current issues with LA traffic in United States and how can we implement smart traffic monitoring system to reduce deaths by road accidents',
  'role': 'assistant',
  'name': 'CityAdmin'},
 {'content': "**Research Report: Addressing Traffic Issues in Los Angeles and Implementing a Smart Traffic Monitoring System**\n\n**Introduction**\nLos Angeles, known for its sprawling urban landscape and vibrant culture, is also infamous for its traffic congestion and road safety challenges. The city's extensive network of highways and streets often experiences severe traffic jams, leading to increased travel times, pollution, and, most concerningly, road accidents. The high volume of vehicles, coupled with complex road networks, contributes to a significant number of traffic-related fatalities and injuries each year. Addressing these issues requires innovative solutions that leverage technology to enhance traffic management and improve road sa

In [23]:
from IPython.display import Markdown

Markdown(output_report.chat_history[-2]['content'])

**Professional Report: Implementation of a Smart Traffic Monitoring System in Los Angeles**

**Project Overview**
Los Angeles, a city renowned for its cultural vibrancy and economic significance, faces persistent challenges with traffic congestion and road safety. The city's complex road networks and high vehicle density contribute to frequent traffic jams and a concerning number of road accidents. To address these issues, the city has embarked on a project to implement a smart traffic monitoring system. This initiative aims to leverage advanced technologies to enhance traffic management, reduce congestion, and improve road safety.

**Implementation Plan**
The implementation of the smart traffic monitoring system involves a structured plan with clearly defined roles for engineers and data analysts. The plan includes six key steps: conducting a comprehensive traffic assessment, designing the system architecture, installing sensors and cameras, developing data processing algorithms, integrating with existing traffic management systems, and monitoring system performance for optimization. Each step is designed to ensure the system's effectiveness in addressing Los Angeles' traffic challenges.

**Data Insights**
Data analysis plays a pivotal role in the implementation process. The Data Analyst is responsible for evaluating historical traffic data to identify patterns and trends, which inform the strategic placement of sensors and cameras. By analyzing peak traffic hours, accident-prone areas, and commuter behavior, the Analyst provides valuable insights that guide decision-making. Additionally, the Analyst develops algorithms to process real-time data, enabling the system to predict congestion and detect incidents effectively.

**System Architecture**
The system architecture, designed by the Engineer, incorporates advanced sensors, cameras, and data analytics platforms. These technologies are selected for their compatibility with existing infrastructure and their ability to collect and process data in real-time. The architecture is designed to be scalable, allowing for future expansions as the city's needs evolve. Integration with current traffic management systems ensures seamless communication and coordinated traffic management.

**Results**
The implementation of the smart traffic monitoring system is expected to yield significant benefits for Los Angeles. By optimizing traffic flow and reducing congestion, the system enhances the efficiency of the city's transportation network. Real-time monitoring capabilities enable quicker responses to traffic incidents, potentially reducing the severity and frequency of accidents. The data-driven approach supports long-term planning and infrastructure development, ensuring the city's transportation system can accommodate future growth.

**Recommendations**
To maximize the impact of the smart traffic monitoring system, it is recommended that the city continue to invest in data analytics capabilities and infrastructure improvements. Ongoing collaboration between engineers and data analysts is essential to ensure the system remains effective and responsive to changing traffic conditions. Additionally, public awareness campaigns can help educate residents on the benefits of the system and encourage safe driving behaviors. By embracing innovative solutions and fostering a culture of safety, Los Angeles can create a sustainable and efficient transportation system for the future.

**Conclusion**
The implementation of a smart traffic monitoring system represents a significant step towards addressing Los Angeles' traffic challenges. By leveraging technology and data-driven insights, the city can enhance its traffic management capabilities, reduce congestion, and improve road safety. As Los Angeles continues to grow and evolve, embracing innovative solutions will be crucial in creating a sustainable and efficient transportation system that meets the needs of its residents.